In [23]:
import pandas as pd
import os
os.chdir("/Users/ericsmac/Documents/MATH300Project/data")
df = pd.read_feather('merged_factors.ft')
ocfa4q = pd.read_feather('ocfa4q.ft')
ocfa4q = ocfa4q[['rk','code','return']]
df = df.merge(ocfa4q, on=['code','rk'], how = 'left')
df

,rk,code,cfo2cl,dta2ev,NNP_SD,ocfa4q,r_nta,roic4q,yoy_s,return
0,1945.0,000004,-0.539357,-0.100045,-0.001195,-0.545248,-0.373964,-1.711607,0.285189,0.114401
1,1945.0,000005,-1.196494,-0.350442,-0.242075,-1.903597,-1.304091,-1.842590,0.416450,0.000000
2,1945.0,000006,-1.569546,1.343961,2.317304,-1.987871,1.056031,0.439823,-2.245958,-0.077917
3,1945.0,000007,1.276299,-0.499103,-0.501018,2.723414,-1.013969,-1.625001,0.792931,-0.009745
4,1945.0,000009,-0.348548,-0.270275,0.043598,-0.184773,-0.233879,-0.161768,-0.313591,0.009129
...,...,...,...,...,...,...,...,...,...,...
5108265,4477.0,688793,-0.855737,-2.272339,-0.464821,-1.076195,-0.982083,-1.067425,-1.258614,-0.138720
5108266,4477.0,688798,0.685225,0.113780,0.200084,0.350019,-0.022114,0.117699,-0.778150,0.026701
5108267,4477.0,688799,0.678994,-0.233143,-1.587911,0.085114,0.300693,0.106962,-0.537688,-0.052172
5108268,4477.0,688800,-0.261622,0.404579,1.854591,-0.065149,0.876517,0.327391,1.797810,-0.047992


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Testing with sample set
df_sample = df.sample(frac=0.2, random_state=42)

2025-12-08 17:14:09.321727: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
import tensorflow as tf

def correlation_loss(y_true, y_pred):
    """Loss = 1 – Pearson correlation"""
    x = y_pred
    y = y_true

    # Normalize
    x -= tf.reduce_mean(x)
    y -= tf.reduce_mean(y)

    # Compute correlation
    corr = tf.reduce_sum(x * y) / (tf.sqrt(tf.reduce_sum(tf.square(x))) *
                                   tf.sqrt(tf.reduce_sum(tf.square(y))) + 1e-8)

    # We want to MINIMIZE the loss, so return 1 - corr
    return 1 - corr

def correlation_metric(y_true, y_pred):
    x = y_pred - tf.reduce_mean(y_pred)
    y = y_true - tf.reduce_mean(y_true)
    corr = tf.reduce_sum(x * y) / (
        tf.sqrt(tf.reduce_sum(tf.square(x))) *
        tf.sqrt(tf.reduce_sum(tf.square(y))) + 1e-8
    )
    return corr



feature_cols = ['cfo2cl', 'dta2ev', 'NNP_SD', 'ocfa4q', 'r_nta', 'roic4q', 'yoy_s']
X = df_sample[feature_cols].values.astype('float32')
y = df_sample['return'].values.astype('float32')

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)   # regression output: predicted return
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=correlation_loss,
    metrics=[correlation_metric]  
)

model.summary()

# ===========================
# 5. Train the model
# ===========================

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=256,
    verbose=1
)

# ===========================
# 6. Evaluate on test set
# ===========================
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test MSE: {test_loss:.6f}, Test MAE: {test_mae:.6f}")

# ===========================
# 7. Make predictions
# ===========================
y_pred = model.predict(X_test_scaled)

from scipy.stats import spearmanr
spearman_corr, pval = spearmanr(y_test, y_pred)
print(f"Test Spearman Corr: {spearman_corr:.4f}")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 32)                256       
                                                                 
 dense_10 (Dense)            (None, 32)                1056      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,345
Trainable params: 1,345
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


2025-12-08 17:34:49.409357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-12-08 17:34:49.491264: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


2555/2555 [==============================] - ETA: 0s - loss: 0.9802 - correlation_metric: 0.0199

2025-12-08 17:35:18.830402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2555/2555 [==============================] - 33s 13ms/step - loss: 0.9802 - correlation_metric: 0.0199 - val_loss: 0.9781 - val_correlation_metric: 0.0218
Epoch 2/50
2555/2555 [==============================] - 33s 13ms/step - loss: 0.9796 - correlation_metric: 0.0205 - val_loss: 0.9779 - val_correlation_metric: 0.0220
Epoch 3/50
2555/2555 [==============================] - 32s 12ms/step - loss: 0.9793 - correlation_metric: 0.0208 - val_loss: 0.9778 - val_correlation_metric: 0.0221
Epoch 4/50
2555/2555 [==============================] - 32s 12ms/step - loss: 0.9794 - correlation_metric: 0.0204 - val_loss: 0.9779 - val_correlation_metric: 0.0221
Epoch 5/50
2555/2555 [==============================] - 32s 12ms/step - loss: 0.9790 - correlation_metric: 0.0210 - val_loss: 0.9783 - val_correlation_metric: 0.0217
Epoch 6/50
2555/2555 [==============================] - 32s 12ms/step - loss: 0.9784 - correlation_metric: 0.0216 - val_loss: 0.9783 - val_correlation_metric: 0.0216
Epoch 7/50
2555

2025-12-08 18:02:19.145467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6386/6386 [==============================] - 16s 2ms/step
Test Spearman Corr: 0.0146


In [31]:


cols = ['cfo2cl','dta2ev','NNP_SD','ocfa4q','r_nta','roic4q','yoy_s']
df_sample['mean'] = df_sample[cols].mean(axis=1)
spearman_corr = df_sample.corr(method='spearman')['return'].sort_values(ascending=False)

print(spearman_corr)



return    1.000000
cfo2cl    0.015017
ocfa4q    0.014627
NNP_SD    0.010474
mean      0.010446
dta2ev    0.005456
r_nta     0.004232
code      0.003795
roic4q   -0.001999
yoy_s    -0.003611
rk       -0.009992
Name: return, dtype: float64
